### 0 Imports & global set-up

In [1]:
import numpy as np
import pandas as pd
from numpy.linalg import inv
from scipy import stats
import matplotlib.pyplot as plt

## Part (a) Unbiasedness & minimal variance of OLS

1. **Unbiasedness**

   In matrix form

   $$
   \widehat\beta_{\text{OLS}}
      =(X^{\mathsf T}X)^{-1}X^{\mathsf T}Y
      =(X^{\mathsf T}X)^{-1}X^{\mathsf T}(X\beta+\varepsilon)
      =\beta+(X^{\mathsf T}X)^{-1}X^{\mathsf T}\varepsilon .
   $$

   With $\mathbb E[\varepsilon\mid X]=0$,

   $$
   \mathbb E[\widehat\beta_{\text{OLS}}\mid X]=\beta .
   $$

2. **Gauss–Markov (BLUE) property**
   Under the homoskedastic/uncorrelated-error assumptions,

   $$
   \operatorname{Var}(\widehat\beta\mid X)=\sigma^{2}(X^{\mathsf T}X)^{-1},
   $$

   and any other *linear* unbiased estimator has covariance matrix minus this one positive-semidefinite ⇒ OLS has the smallest variance component-wise.

## Generate one synthetic data-set

In [2]:
# problem constants
np.random.seed(42)
n     = 500                   # sample size
mu    = np.array([0,1,1,2,2]) # E[X]
Sigma = np.eye(5)             # Cov[X]
beta  = np.array([2,-3,2,1,6,-2])  # β0 … β5
σ2    = 1.0

# draw predictors and build design matrix
X_raw = np.random.multivariate_normal(mu, Sigma, size=n)
X     = np.hstack([np.ones((n,1)), X_raw])

# generate homoskedastic noise and response
eps = np.random.normal(0, σ2**0.5, size=n)
y   = X @ beta + eps


## Compute one OLS estimate

In [3]:
beta_hat = inv(X.T @ X) @ X.T @ y
pd.Series(beta_hat, index=[f"β{i}" for i in range(6)])

β0    2.135557
β1   -2.988848
β2    1.990775
β3    0.957853
β4    5.950870
β5   -1.981647
dtype: float64

## Monte-Carlo loop (i guess)

In [4]:
n_rep = 10_000
beta_hats = np.empty((n_rep, 6))

for r in range(n_rep):
    X_raw = np.random.multivariate_normal(mu, Sigma, size=n)
    X     = np.hstack([np.ones((n,1)), X_raw])
    eps   = np.random.normal(0, σ2**0.5, size=n)
    y     = X @ beta + eps
    beta_hats[r] = inv(X.T @ X) @ X.T @ y

# empirical mean & covariance
mean_hat = beta_hats.mean(axis=0)
cov_hat  = np.cov(beta_hats, rowvar=False)

display(pd.DataFrame([mean_hat], columns=[f"β{i}" for i in range(6)]))
print("\nEmpirical covariance of β̂ (first 4×4 block):")
display(pd.DataFrame(cov_hat[:4,:4],
                     index=[f"β{i}" for i in range(4)],
                     columns=[f"β{i}" for i in range(4)]))

,β0,β1,β2,β3,β4,β5
0,1.997303,-3.000386,2.000537,1.000252,5.999982,-1.998771



Empirical covariance of β̂ (first 4×4 block):


,β0,β1,β2,β3
β0,0.022567,-0.000049,-0.002019,-0.002065
β1,-0.000049,0.001998,-0.000022,-0.000007
β2,-0.002019,-0.000022,0.002027,0.000009
β3,-0.002065,-0.000007,0.000009,0.002050


**Interpretation**

* The empirical means lie extremely close to the true coefficients
  $(2,-3,2,1,6,-2)$ ⇒ confirmation of unbiasedness.
* The empirical covariance matrix matches the theoretical
  $\sigma^{2}(X^{\mathsf T}X)^{-1}$ once averaged across replication

## Two noise scenarios with heteroskedastic / heavy-tailed errors

### 1.  $\varepsilon_i \sim \mathcal N\!\bigl(0,\lVert X_i\rVert^{2}\bigr)$  ▶ Run


In [5]:
def beta_hat_hetero():
    X_raw = np.random.multivariate_normal(mu, Sigma, size=n)
    X     = np.hstack([np.ones((n,1)), X_raw])
    sig_i = np.linalg.norm(X_raw, axis=1)       # std-dev depends on ‖X_i‖
    eps   = np.random.normal(0, sig_i)
    y     = X @ beta + eps
    return inv(X.T @ X) @ X.T @ y

beta_hetero = np.vstack([beta_hat_hetero() for _ in range(n_rep)])
print("Bias (heteroskedastic case):")
display(pd.Series(beta_hetero.mean(0) - beta,
                  index=[f"β{i}" for i in range(6)]))

Bias (heteroskedastic case):


β0    0.006448
β1    0.004288
β2    0.000760
β3   -0.001567
β4   -0.002338
β5   -0.001143
dtype: float64

### 2. Heavy-tailed Cauchy noise $\varepsilon_i\sim t_{1}(0,1)$  ▶ Run

In [6]:

def beta_hat_cauchy():
    X_raw = np.random.multivariate_normal(mu, Sigma, size=n)
    X     = np.hstack([np.ones((n,1)), X_raw])
    eps   = stats.cauchy.rvs(size=n)   # df = 1
    y     = X @ beta + eps
    return inv(X.T @ X) @ X.T @ y

beta_cauchy = np.vstack([beta_hat_cauchy() for _ in range(n_rep)])
print("Std-error explosion with Cauchy noise (MSE):")
mse = ((beta_cauchy - beta)**2).mean(0)
display(pd.Series(mse, index=[f"β{i}" for i in range(6)]))


Std-error explosion with Cauchy noise (MSE):


β0    193317.743342
β1      2256.837843
β2      7164.076689
β3      1365.050449
β4      9633.882544
β5      2678.172742
dtype: float64

*With heteroskedastic errors* OLS is still unbiased but its variance is larger;
*with Cauchy errors* the second moment of the noise is infinite, causing the OLS variance (and MSE) to blow up.